In [109]:
current_dir = '/Users/shimsomin/project-template'

In [110]:
import sys
sys.path.append('../')
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
import krwordrank
import pandas as pd

In [111]:
def get_texts_scores(fname):
    # 튜토리얼에서 이용하는 `fname` 파일은 영화평과 평점이 \t 으로 구분된 two column tsv 파일
    with open(fname, encoding='utf-8') as f:
        docs = [doc.lower().replace('\n','').split('\t') for doc in f]
        docs = [doc for doc in docs if len(doc) == 2]

        if not docs:
            return [], []

        texts, scores = zip(*docs)
        return list(texts), list(scores)

In [112]:
# La La Land
fname = current_dir + '/data/134963.txt'
texts, scores = get_texts_scores(fname)

with open(fname, encoding="utf-8") as f:
    for _ in range(5):
        print(next(f).strip())

시사회에서 보고왔습니다동화와 재즈뮤지컬의 만남! 지루하지않고 재밌습니다	9
사랑과 꿈, 그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다. 배우들 연기는 두말할것없고	10
지금껏 영화 평가 해본 적이 없는데 진짜..최고네요! 색감. 스토리.음악.연기 모두ㅜㅜ최고입니다!!!!	10
방금 시사회 보고 왔어요~ 배우들 매력이 눈을 뗄 수가 없게 만드네요. 한편의 그림 같은 장면들도 많고, 음악과 춤이 눈과 귀를 사로 잡았어요. 한번 더 보고 싶네요.	10
초반부터 끝까지 재미있게 잘보다가 결말에서 고국마 왕창먹음...힐링 받는 느낌들다가 막판에 기분 잡쳤습니다. 마치 감독이 하고싶은 말은 "너희들이 원하는 결말은 이거지? 하지만 현실은 이거다!!" 라고 말하고 싶었나보군요	1


In [113]:
# 정규화된 텍스트 저장
with open(current_dir+'/data/134963_norm.txt', 'w', encoding='utf-8') as f:
    for text, score in zip(texts, scores):
        text = normalize(text, english=True, number=True)
        f.write('%s\t%s\n' % (text, str(score)))

In [114]:
# La La Land
fname = current_dir+'/data/134963_norm.txt'
texts, scores = get_texts_scores(fname)

In [115]:
wordrank_extractor = KRWordRank(
    min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10, # 단어의 최대 길이
    verbose = True
    )

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

scan vocabs ... 
num vocabs = 13822
done = 9 Early stopped.


In [38]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10]:
    print('%8s:\t%.4f' % (word, r))

      영화:	201.9608
      너무:	81.7410
      정말:	40.8016
      음악:	40.3295
     마지막:	38.9302
     뮤지컬:	23.1365
      최고:	22.0345
      사랑:	20.5910
      영상:	20.4099
      아름:	20.3295


## 영화 리뷰 - 영화 id 데이터세트 만들기

In [120]:
movies_data = pd.read_csv(current_dir+'/data/movies_data.csv')

In [121]:
movies_data[movies_data['title'].isin(['라라랜드','신세계'])]

,Unnamed: 0,title,type,open_year,rate,running_time,genre1,genre2,genre3,actor,director,summary
38,38,라라랜드,Movie,2016,8.7,2시간 8분,코미디,드라마,음악,"Ryan Gosling, Emma Stone, John Legend, Rosemar...",Damien Chazelle,차들로 빽빽이 들어찬 LA의 고속도로. 거북이 걸음이던 도로가 뚫리기 시작하지만 미...
356,356,신세계,Movie,2013,8.1,2시간 14분,스릴러,액션,범죄,"Lee Jung-jae, Choi Min-sik, Hwang Jung-min, Pa...",Park Hoon-jung,경찰청 수사기획과 강과장은 국내 최대 범죄 조직 골드문이 기업형 조직으로 세력이 확...


In [123]:
lala_text = texts

In [124]:
# review_movie_id = pd.DataFrame(columns=['movie_id', 'review'])
lalaland = pd.DataFrame(columns=['movie_id', 'review'])
sinsegae = pd.DataFrame(columns=['movie_id', 'review'])
lalaland['review'] = lala_text
lalaland['movie_id'] = 38

In [53]:
fname = current_dir+'/data/91031.txt'

sin_texts, scores = get_texts_scores(fname)
with open(fname.replace('.txt', '_norm.txt'), 'w', encoding='utf-8') as f:
    for text, score in zip(sin_texts, scores):
        text = normalize(text, english=True, number=True)
        f.write('%s\t%s\n' % (text, str(score)))

In [125]:
# 신세계
fname = current_dir+'/data/91031_norm.txt'
sin_texts, scores = get_texts_scores(fname)

In [126]:
sinsegae['review'] = sin_texts
sinsegae['movie_id'] = 356

In [129]:
review_movie_id = pd.concat([lalaland, sinsegae]).reset_index(drop=True)

In [130]:
review_movie_id

,movie_id,review
0,38,시사회에서 보고왔습니다동화와 재즈뮤지컬의 만남 지루하지않고 재밌습니다
1,38,사랑과 꿈 그 흐름의 아름다움을 음악과 영상으로 최대한 담아놓았다 배우들 연기는 두...
2,38,지금껏 영화 평가 해본 적이 없는데 진짜 최고네요 색감 스토리 음악 연기 모두 최고입니다
3,38,방금 시사회 보고 왔어요 배우들 매력이 눈을 뗄 수가 없게 만드네요 한편의 그림 같...
4,38,초반부터 끝까지 재미있게 잘보다가 결말에서 고국마 왕창먹음 힐링 받는 느낌들다가 막...
...,...,...
35500,356,잼있겠다 7번방 9점 베를린 8점 박수 6점 박수는 솔직히 너무했어요 귀신 사람몸속...
35501,356,무간도를 능가하지는 못했지만 무간도급 재미의 영화 일단 시나리오가 찰지다 뭔가 쫀득...
35502,356,재미있을것 같네요
35503,356,부당거래의 후속작을 베를린이라고 생각했으나 바로 이 영화였다 박훈정의 신세계무간도의...


In [131]:
review_movie_id.to_csv(current_dir+'/data/review_movie_id.csv', index=False)

## 불용어 정의하기

In [107]:
top_keywords = []
fnames = [current_dir+'/data/134963_norm.txt',
          current_dir+'/data/91031_norm.txt',
          current_dir+'/data/99714_norm.txt']

for fname in fnames:
    
    texts, scores = get_texts_scores(fname)
    
    wordrank_extractor = KRWordRank(
        min_count=5, max_length=10, verbose=False)
    
    keywords, rank, graph = wordrank_extractor.extract(
        texts, beta, max_iter)
    
    top_keywords.append(
        sorted(keywords.items(),
               key=lambda x:x[1],
               reverse=True)[:100]
    )

In [85]:
movie_names = ['라라랜드', '신세계', '엑스맨']
for k in range(100):
    
    message = '  --  '.join(
        ['%8s (%.3f)' % (top_keywords[i][k][0],top_keywords[i][k][1])
         for i in range(3)])
    
    print(message)
    

      영화 (201.961)  --        영화 (145.840)  --       엑스맨 (112.845)
      너무 (81.741)  --       황정민 (99.563)  --        영화 (65.482)
      정말 (40.802)  --        연기 (89.988)  --        정말 (42.081)
      음악 (40.329)  --        정말 (74.653)  --        진짜 (40.935)
     마지막 (38.930)  --        진짜 (64.803)  --       시리즈 (40.835)
     뮤지컬 (23.137)  --        최고 (54.792)  --        너무 (38.929)
      최고 (22.034)  --        너무 (51.963)  --        재밌 (34.045)
      사랑 (20.591)  --       이정재 (46.503)  --        재미 (30.454)
      영상 (20.410)  --       무간도 (36.144)  --        최고 (28.824)
      아름 (20.329)  --       배우들 (34.579)  --        기대 (24.799)
      꿈을 (20.300)  --        재밌 (28.948)  --       스토리 (24.206)
     여운이 (19.457)  --       스토리 (26.741)  --        역시 (23.917)
      진짜 (19.282)  --        한국 (26.367)  --        보고 (19.043)
      노래 (18.788)  --       신세계 (24.480)  --        액션 (17.983)
      보고 (18.482)  --        대박 (23.847)  --        그냥 (17.412)
      좋았 (17.678)  --       최민식 (19.8

In [86]:
keyword_counter = {}
for keywords in top_keywords:
    words, ranks = zip(*keywords)
    for word in words:
        keyword_counter[word] = keyword_counter.get(word, 0) + 1

common_keywords = {word for word, count in keyword_counter.items() if count == 3}
len(common_keywords)

43

In [96]:
str(common_keywords)

"{'영화', '봤는데', '보고', '사람', '아주', '그리고', '가장', '생각', '추천', '보는', '좋아', '처음', '봤습니다', '그래도', '아니', '진짜', '지루', '스토리', '느낌', '이렇게', '평점', '마지막', '기대', '이런', '있는', '내용', '최고', '정말', '별로', '재미', '많이', '하지만', '한번', '하나', '재밌', '내가', '좋았', '너무', '다시', '그냥', '보면', '모두', '조금'}"

In [88]:
selected_top_keywords = []
for keywords in top_keywords:
    selected_keywords = []
    for word, r in keywords:
        if word in common_keywords:
            continue
        selected_keywords.append((word, r))
    selected_top_keywords.append(selected_keywords)

In [89]:
def get_from_list(l, i, default=('', 0)):
    if len(l) <= i:
        return default
    else:
        return l[i]

In [94]:
for k in range(48 - len(common_keywords) ):
    message = '  --  '.join(
        ['%8s (%.3f)' % get_from_list(selected_top_keywords[i], k) for i in range(3)])
    
    print(message)
    

      음악 (40.329)  --       황정민 (99.563)  --       엑스맨 (112.845)
     뮤지컬 (23.137)  --        연기 (89.988)  --       시리즈 (40.835)
      사랑 (20.591)  --       이정재 (46.503)  --        역시 (23.917)
      영상 (20.410)  --       무간도 (36.144)  --        액션 (17.983)
      아름 (20.329)  --       배우들 (34.579)  --      브라이언 (16.782)


In [103]:
def define_stopwords(path):
    SW = set()

    with open(path, encoding="utf-8") as f:
        for word in f:
            SW.add(word.strip())
            
    SW.add("있어요")
    SW.add("대한")
    SW.add("합니다")
    SW.add("하는")
    
    return SW

In [104]:
SW = define_stopwords(current_dir + "/data/stopwords.txt")